## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os
from pathlib import Path
import unisens
from scipy.signal import resample
import pickle
from imblearn.ensemble import BalancedRandomForestClassifier
import os
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold

from imblearn.ensemble import BalancedRandomForestClassifier
from random import sample

from imblearn.ensemble import BalancedRandomForestClassifier

from tqdm import tqdm

import warnings
from sklearn import __version__ as skl_version
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
import scipy.signal as signal

## Initialize label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = numpy.load('classes.npy')

## Load RNN Model

In [ ]:
filepath = "RNN.keras"

rnn = tf.keras.models.load_model(filepath)

## Load CNN Model

In [ ]:
filepath = "CNN.keras"

rnn = tf.keras.models.load_model(filepath)

## Load RF Model

In [ ]:
# Load the model
with open('rf_model.pkl', 'rb') as f:
    random_forest = pickle.load(f)

## extract_features
A function the extract the relevant features from the data for the model.

In [ ]:
def extract_features(xyz, sample_rate=64):
    """Extract commonly used HAR time-series features. xyz is a window of shape (N,3)"""

    feats = {}

    x, y, z = xyz.T

    feats['xmin'], feats['xq25'], feats['xmed'], feats['xq75'], feats['xmax'] = np.quantile(x, (0, .25, .5, .75, 1))
    feats['ymin'], feats['yq25'], feats['ymed'], feats['yq75'], feats['ymax'] = np.quantile(y, (0, .25, .5, .75, 1))
    feats['zmin'], feats['zq25'], feats['zmed'], feats['zq75'], feats['zmax'] = np.quantile(z, (0, .25, .5, .75, 1))

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        # xy, xy, zx correlation
        feats['xycorr'] = np.nan_to_num(np.corrcoef(x, y)[0, 1])
        feats['yzcorr'] = np.nan_to_num(np.corrcoef(y, z)[0, 1])
        feats['zxcorr'] = np.nan_to_num(np.corrcoef(z, x)[0, 1])

    v = np.linalg.norm(xyz, axis=1)

    feats['min'], feats['q25'], feats['med'], feats['q75'], feats['max'] = np.quantile(v, (0, .25, .5, .75, 1))

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        # 1s autocorrelation
        feats['corr1s'] = np.nan_to_num(np.corrcoef(v[:-sample_rate], v[sample_rate:]))[0, 1]

    # Angular features
    feats.update(angular_features(xyz, sample_rate))

    # Spectral features
    feats.update(spectral_features(v, sample_rate))

    # Peak features
    feats.update(peak_features(v, sample_rate))

    return feats


def spectral_features(v, sample_rate):
    """ Spectral entropy, 1st & 2nd dominant frequencies """

    feats = {}

    # Spectrum using Welch's method with 3s segment length
    # First run without detrending to get the true spectrum
    freqs, powers = signal.welch(v, fs=sample_rate,
                                 nperseg=3 * sample_rate,
                                 noverlap=2 * sample_rate,
                                 detrend=False,
                                 average='median')

    with np.errstate(divide='ignore', invalid='ignore'):  # ignore div by 0 warnings
        feats['pentropy'] = np.nan_to_num(stats.entropy(powers + 1e-16))

    # Spectrum using Welch's method with 3s segment length
    # Now do detrend to focus on the relevant freqs
    freqs, powers = signal.welch(v, fs=sample_rate,
                                 nperseg=3 * sample_rate,
                                 noverlap=2 * sample_rate,
                                 detrend='constant',
                                 average='median')

    peaks, _ = signal.find_peaks(powers)
    peak_powers = powers[peaks]
    peak_freqs = freqs[peaks]
    peak_ranks = np.argsort(peak_powers)[::-1]
    if len(peaks) >= 2:
        feats['f1'] = peak_freqs[peak_ranks[0]]
        feats['f2'] = peak_freqs[peak_ranks[1]]
        feats['p1'] = peak_powers[peak_ranks[0]]
        feats['p2'] = peak_powers[peak_ranks[1]]
    elif len(peaks) == 1:
        feats['f1'] = feats['f2'] = peak_freqs[peak_ranks[0]]
        feats['p1'] = feats['p2'] = peak_powers[peak_ranks[0]]
    else:
        feats['f1'] = feats['f2'] = 0
        feats['p1'] = feats['p2'] = 0

    return feats


def peak_features(v, sample_rate):
    """ Features of the signal peaks. A proxy to step counts. """

    feats = {}
    u = butterfilt(v, (.6, 5), fs=sample_rate)
    peaks, peak_props = signal.find_peaks(u, distance=0.2 * sample_rate, prominence=0.25)
    feats['numPeaks'] = len(peaks)
    if len(peak_props['prominences']) > 0:
        feats['peakPromin'] = np.median(peak_props['prominences'])
    else:
        feats['peakPromin'] = 0

    return feats


def angular_features(xyz, sample_rate):
    """ Roll, pitch, yaw.
    Hip and Wrist Accelerometer Algorithms for Free-Living Behavior
    Classification, Ellis et al.
    """

    feats = {}

    # Raw angles
    x, y, z = xyz.T

    roll = np.arctan2(y, z)
    pitch = np.arctan2(x, z)
    yaw = np.arctan2(y, x)

    feats['avgroll'] = np.mean(roll)
    feats['avgpitch'] = np.mean(pitch)
    feats['avgyaw'] = np.mean(yaw)
    feats['sdroll'] = np.std(roll)
    feats['sdpitch'] = np.std(pitch)
    feats['sdyaw'] = np.std(yaw)

    # Gravity angles
    xyz = butterfilt(xyz, 0.5, fs=sample_rate)

    x, y, z = xyz.T

    roll = np.arctan2(y, z)
    pitch = np.arctan2(x, z)
    yaw = np.arctan2(y, x)

    feats['rollg'] = np.mean(roll)
    feats['pitchg'] = np.mean(pitch)
    feats['yawg'] = np.mean(yaw)

    return feats


def butterfilt(x, cutoffs, fs, order=10, axis=0):
    nyq = 0.5 * fs
    if isinstance(cutoffs, tuple):
        hicut, lowcut = cutoffs
        if hicut > 0:
            btype = 'bandpass'
            Wn = (hicut / nyq, lowcut / nyq)
        else:
            btype = 'low'
            Wn = lowcut / nyq
    else:
        btype = 'low'
        Wn = cutoffs / nyq
    sos = signal.butter(order, Wn, btype=btype, analog=False, output='sos')
    y = signal.sosfiltfilt(sos, x, axis=axis)
    return y


def get_feature_names():
    """ Hacky way to get the list of feature names """

    feats = extract_features(np.zeros((1000, 3)), 100)
    return list(feats.keys())

## extract_windows
A function to create windows. We can also change the frequency of the windows by downsampling.

In [ ]:
def extract_windows(data, winsize=90, input_frequency=64, output_frequency=64):
    """
    Extracts sliding windows from time series data and labels for classification tasks.

    Args:
        data (pd.DataFrame): Time series DataFrame with 'x', 'y', 'z' (accelerometer data) and 'label'.
                             Index should be a datetime-like index.
        winsize (int): Window size in seconds.
        input_frequency (int): Sampling frequency of input data in Hz.
        output_frequency (int): Desired output frequency in Hz (must be a divisor of input_frequency).

    Returns:
        X (np.ndarray): Shape (n_samples, output_samples, 3), accelerometer windows.
        Y (np.ndarray): Shape (n_samples,), most frequent label per window.
    """
    # Calculate window size in samples and target output samples
    window_samples = winsize * input_frequency
    output_samples = winsize * output_frequency  # Expected downsampled length

    X, Y = [], []

    # Sliding window extraction
    for start in range(0, len(data) - window_samples + 1, window_samples):
        window = data.iloc[start:start + window_samples]

        # Skip if missing values exist
        if window.isna().any().any() or len(window) != window_samples:
            continue

        # Extract and resample accelerometer data
        x = window[['x', 'y', 'z']].to_numpy()
        x = resample(x, output_samples)  # Resample to match output frequency

        # Extract the most frequent label (mode)
        y = window['label'].mode().iloc[0]

        X.append(x)
        Y.append(y)

    X = np.stack(X) if X else np.empty((0, output_samples, 3))
    Y = np.array(Y) if Y else np.empty((0,))

    return X, Y

## normalize
Normalize the data to make it better for generalization

In [ ]:
def normalize(data, feature_cols):
    """
    Normalizes the specified feature columns of a DataFrame using Min-Max scaling.

    Args:
        data (pd.DataFrame): A DataFrame containing the features to be normalized.
        feature_cols (list of str): List of column names in `data` to normalize.
        
    Returns:
        pd.DataFrame: A new DataFrame with the specified columns normalized to the range [0, 1].
        The original `data` DataFrame is modified in place.
        
    Notes:
        - Min-Max normalization scales each feature column to the range [0, 1] based on 
          the minimum and maximum values of the column.
        - This transformation is often used to prepare data for machine learning models 
          that are sensitive to feature magnitudes.
    """
    scaler = MinMaxScaler()
    data[feature_cols] = scaler.fit_transform(data[feature_cols])
    return data

## Load the Data

In [ ]:
def import_and_process_imu_data(directory, accel_freq=64):
    # Define the directory containing the IMU data
    imu_directory = os.path.join(directory, 'imu')
    imu_path = Path(imu_directory)

    # Flags to track processed hands
    l_hand = False
    r_hand = False
    
    # DataFrames to hold IMU data for left and right wrists
    imu_data_l = None
    imu_data_r = None
    
    # Iterate over all XML files in the directory
    for file_path in imu_path.rglob("*.xml"):
        # Get the directory containing the unisens.xml
        unisens_directory = os.path.dirname(file_path)
        u = unisens.Unisens(unisens_directory)

        # Extract raw accelerometer and gyroscope data
        raw_accel_data = u['acc.bin'].get_data()
        metadata = u['customAttributes']
        
        # Extract metadata attributes
        metadata_dict = vars(metadata)
        sensorLocation = metadata_dict['attrib']['sensorLocation']

        # Skip processing if data for the same wrist has already been processed
        if sensorLocation == 'left_wrist' and l_hand:
            print(f"Skipping already processed left wrist data in {file_path}")
            continue
        if sensorLocation == 'right_wrist' and r_hand:
            print(f"Skipping already processed right wrist data in {file_path}")
            continue

        # Parse imu_timestampStartUTC
        imu_timestampStartUTC = pd.to_datetime(metadata_dict['attrib']['timestampStartUTC'], format='%Y-%m-%dT%H:%M:%S.%f')
        timezone_offset_seconds = int(metadata_dict['attrib']['timeZoneOffset'])
        imu_timestampStartUTC += pd.Timedelta(seconds=timezone_offset_seconds)

        # Generate timestamps efficiently using pd.date_range
        num_samples = len(raw_accel_data[0])
        timestamps = pd.date_range(start=imu_timestampStartUTC, periods=num_samples, freq=f"{int(1 / accel_freq * 1e3)}ms")
        
        # Prepare IMU data for the left wrist
        if sensorLocation == 'left_wrist':
            l_hand = True
            imu_data_l = pd.DataFrame({
                'timestamp': timestamps,
                'x': raw_accel_data[0],  # accelerometer x
                'y': raw_accel_data[1],  # accelerometer y
                'z': raw_accel_data[2],  # accelerometer z
            })
        
        # Prepare IMU data for the right wrist
        if sensorLocation == 'right_wrist':
            r_hand = True
            imu_data_r = pd.DataFrame({
                'timestamp': timestamps,
                'x': raw_accel_data[0],  # accelerometer x
                'y': raw_accel_data[1],  # accelerometer y
                'z': raw_accel_data[2],  # accelerometer z
            })
    
        if sensorLocation == 'left_wrist':
            imu_data_l['timestamp'] = pd.to_datetime(imu_data_l['timestamp'])
        
        if sensorLocation == 'right_wrist':
            imu_data_r['timestamp'] = pd.to_datetime(imu_data_r['timestamp'])
    
    
    
    # Return the processed IMU data
    return imu_data_l, imu_data_r

## Apply the RNN Model

In [ ]:
# Load the IMU data
imu_data_l, imu_data_r = import_and_process_imu_data("/Volumes/T7/eXprt-backup/05022024/data/MP-P002/")
imu_data_l.set_index('timestamp', inplace=True)
imu_data_r.set_index('timestamp', inplace=True)

# Normalize the accelerometer data
imu_data_l = normalize(imu_data_l, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_l = extract_windows(imu_data_l, winsize=90, input_frequency=64, output_frequency=64)

# Apply the model to the extracted windows
predictions_l_prob = rnn.predict(X_l)
predictions_l = label_encoder.inverse_transform(predictions_l_prob.argmax(axis=1))

# Normalize the accelerometer data
imu_data_r = normalize(imu_data_r, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_r = extract_windows(imu_data_r, winsize=90, input_frequency=64, output_frequency=64)

# Apply the model to the extracted windows
predictions_r_prob = rnn.predict(X_r)
predictions_r = label_encoder.inverse_transform(predictions_r_prob.argmax(axis=1))

# Compare the two wrists
print(predictions_l)
print(predictions_r)

min_length = min(len(predictions_l), len(predictions_r))
predictions_l = predictions_l[:min_length]
predictions_r = predictions_r[:min_length]

# Calculate the percentage of time the model predicts the same activity for both wrists
same_activity = np.mean(predictions_l == predictions_r)

print(f"Percentage of time the model predicts the same activity for both wrists: {same_activity:.2%}")

predictions_l_prob = predictions_l_prob[:min_length]
predictions_r_prob = predictions_r_prob[:min_length]

# Calculate probality similarity
l2 = np.linalg.norm(predictions_l_prob - predictions_r_prob)

print(f"Probability similarity between the two wrists: {l2:.2f}")

## Apply the CNN Model

In [ ]:
# Load the IMU data
imu_data_l, imu_data_r = import_and_process_imu_data("/Volumes/T7/eXprt-backup/05022024/data/MP-P002/")
imu_data_l.set_index('timestamp', inplace=True)
imu_data_r.set_index('timestamp', inplace=True)

# Normalize the accelerometer data
imu_data_l = normalize(imu_data_l, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_l = extract_windows(imu_data_l, winsize=90, input_frequency=64, output_frequency=64)

# Apply the model to the extracted windows
predictions_l_prob = cnn.predict(X_l)
predictions_l = label_encoder.inverse_transform(predictions_l_prob.argmax(axis=1))

# Normalize the accelerometer data
imu_data_r = normalize(imu_data_r, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_r = extract_windows(imu_data_r, winsize=90, input_frequency=64, output_frequency=64)

# Apply the model to the extracted windows
predictions_r_prob = cnn.predict(X_r)
predictions_r = label_encoder.inverse_transform(predictions_r_prob.argmax(axis=1))

# Compare the two wrists
print(predictions_l)
print(predictions_r)

min_length = min(len(predictions_l), len(predictions_r))
predictions_l = predictions_l[:min_length]
predictions_r = predictions_r[:min_length]

# Calculate the percentage of time the model predicts the same activity for both wrists
same_activity = np.mean(predictions_l == predictions_r)

print(f"Percentage of time the model predicts the same activity for both wrists: {same_activity:.2%}")

predictions_l_prob = predictions_l_prob[:min_length]
predictions_r_prob = predictions_r_prob[:min_length]

# Calculate probality similarity
l2 = np.linalg.norm(predictions_l_prob - predictions_r_prob)

print(f"Probability similarity between the two wrists: {l2:.2f}")

## Apply the Random Forest Model

In [ ]:
# Load the IMU data
imu_data_l, imu_data_r = import_and_process_imu_data("/Volumes/T7/eXprt-backup/05022024/data/MP-P002/")
imu_data_l.set_index('timestamp', inplace=True)
imu_data_r.set_index('timestamp', inplace=True)

# Normalize the accelerometer data
imu_data_l = normalize(imu_data_l, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_l = extract_windows(imu_data_l, winsize=90, input_frequency=64)
X_l_feats = pd.DataFrame([extract_features(x) for x in X_l])

# Apply the model to the extracted windows
predictions_l_prob = random_forest.predict_proba(X_l_feats)

# Normalize the accelerometer data
imu_data_r = normalize(imu_data_r, ['x', 'y', 'z'])

# Extract windows from the accelerometer data
X_r = extract_windows(imu_data_r, winsize=90, input_frequency=64)
X_r_feats = pd.DataFrame([extract_features(x) for x in X_r])

# Apply the model to the extracted windows
predictions_r_prob = random_forest.predict_proba(X_r_feats)

# Ensure both arrays have the same length
min_length = min(len(predictions_l_prob), len(predictions_r_prob))
predictions_l_prob = predictions_l_prob[:min_length]
predictions_r_prob = predictions_r_prob[:min_length]

# Compare the two wrists
print(predictions_l_prob)
print(predictions_r_prob)

predictions_l = random_forest.predict(X_l_feats)[:min_length]
predictions_r = random_forest.predict(X_r_feats)[:min_length]

# Compare predictions
print(predictions_l)
print(predictions_r)

# Calculate the percentage of time the model predicts the same activity for both wrists
same_activity = np.mean(predictions_l == predictions_r)

print(f"Percentage of time the model predicts the same activity for both wrists: {same_activity:.2%}")

# Calculate probability similarity
l2 = np.linalg.norm(predictions_l_prob - predictions_r_prob)

print(f"Probability similarity between the two wrists: {l2:.2f}")